# Persistence and ORMs

## Databases

90% of applications have a need to store data. Storing data simply in files becomes difficult and inefficient after a while. After a while the file containing the data might start getting bigger, and so you would need to start implementing features to speed up the file reading and writing, which in a nutshell is just what the databases do.

Databases are purpose built applications for storing and retrieving data.

## CAP theorem

CAP theorem applies to distributed data stores. According to CAP, a distributed database can only guarantee 2 of these traits:

```mermaid
mindmap
  root(CAP)
    Consistency
    Availability
    Partition tolerance
```

Consistency - all reads receive the most recently written data.
Availability - database is always ready for use.
Partition tolerance - data storage can be split into multiple shards or replicas.

## Migrations

```mermaid
mindmap
  root(Database migrations)
    Database first
    Code first
    Model first
```

### Benefits of using migrations

Pros: 
- Speeds up the development, since you can generate part of reflected code from the existing code.
- Ensures consistency between database and code.
- Allows to change either the code or database easier.

Cons:
- If it does not work properly out of the box, then it might end up being a huge time waste.

### Database first

Database first Migrations is a development approach where you design your database structure first. It can be done using tools like MySQL Workbench or SQL Server Management Studio, or just running queries via CLI. The point is that the database schema must be created before you start generating code. Once the database is available, then you generate the code, such as entity classes, based on the database schema created. This approach is particularly useful for projects where you need to work with an existing database or want to ensure your code aligns with a predefined database structure.

### Model first

Model-First Migrations is a development approach where you design your database schema using a visual modeling tool, designer tool within an ORM framework like Entity Framework, or some intermediate modeling language.

After creating the model, you generate the corresponding database schema and code, including entity classes and relationships. This approach is OK when you first start out by designing your database visually.

### Code first

Code first migrations allows to generate database structure according to the code that is written. It is a typical feature of ORM frameworks like Entity Framework.

Pros:
- Fastest to work with.
- Changes are neatly versioned.
- Code always (mostly) work.

Cons:
- Tough luck if the migrations library does not support some database feature needed.


## ORM vs no ORM

ORM stands for object-relational-mapper. Using ORM means that you interact with the database via classes and methods instead of working directly with the queries against the database. ORMs simplifies the usage of databases at the cost of reduced flexibility.

Pros of using ORM:
- No need to deal with SQL query statements.
- Easier to refactor code, since IDE can track down all the references.
- ORM takes care of lots of boilerplate code.

Cons of using ORM:
- You can only use the database features that the ORM framework supports.
- Since query statements are generated by ORM, they can be suboptimal performance wise.

## Connection strings

Connection strings specify how to connect and use the data source.

Example connection string:
>`User ID=root;Password=myPassword;Host=localhost;Port=5432;Database=myDataBase;Pooling=true;Min Pool Size=0;Max Pool Size=100;Connection Lifetime=0;`

Connection strings can contain usernames, passwords, host address and various other parameters. Because of this it is not recommended to hard code, but rather to create a mechanism for them to be passed via some means of configuration.

Connections strings differ based on database connection driver used. Same connection string that worked for one relational database, may not work for other relation database. Always check the reference like https://www.connectionstrings.com/ to be sure how to assemble connection string correctly for your case.

## Persistence in .NET

.NET has libraries to work with most of the commonly available databases. This section will take a look at 2 libraries for working with relational databases: `Dapper` and `EntityFramework`.

For the sake of examples, we'll use PostgreSQL running in a Docker container.

To start a container run this command:
```bash
docker run --name postgres-dev -e POSTGRES_PASSWORD=devpassword -e POSTGRES_DB=example -p 5432:5432 -d postgres:15
```

After being done with the examples, the container can be removed by running:
```bash
docker rm postgres-dev
```

Connection strings in the following examples are supposed to match what is running in the container started with the command above.

### Creating Tables

In [ ]:
#r "nuget:Npgsql"

In [ ]:
using Npgsql;

var connectionString = "Host=localhost;Port=5432;Database=example;Username=postgres;Password=devpassword;";

var connection = new NpgsqlConnection(connectionString);
connection.Open();

// Drop tables if they exist (useful for resetting)
var dropTables = @"
    DROP TABLE IF EXISTS subjects CASCADE;
    DROP TABLE IF EXISTS courses CASCADE;";

using (var cmd = new NpgsqlCommand(dropTables, connection))
{
    cmd.ExecuteNonQuery();
}

// Create courses table
var createCourses = @"
    CREATE TABLE courses (
        id SERIAL PRIMARY KEY,
        course_name VARCHAR(100) NOT NULL,
        course_description TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );";

using (var cmd = new NpgsqlCommand(createCourses, connection))
{
    cmd.ExecuteNonQuery();
}

// Create subjects table
var createSubjects = @"
    CREATE TABLE subjects (
        id SERIAL PRIMARY KEY,
        subject_name VARCHAR(100) NOT NULL,
        subject_description TEXT,
        course_id INT REFERENCES courses(id) ON DELETE CASCADE,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );";

using (var cmd = new NpgsqlCommand(createSubjects, connection))
{
    cmd.ExecuteNonQuery();
}

Console.WriteLine("Tables created successfully!");


### Dapper

In [ ]:
// Define the classes that represent the schema:

public class Course
{
    public int Id { get; set; }
    public string CourseName { get; set; }
    public string CourseDescription { get; set; }
    public DateTime CreatedAt { get; set; }
}

public class Subject
{
    public int Id { get; set; }
    public string SubjectName { get; set; }
    public string SubjectDescription { get; set; }
    public int CourseId { get; set; }
    public DateTime CreatedAt { get; set; }
}

Add the `Dapper` package from nuget:

In [ ]:
#r "nuget:Dapper"

In [ ]:
using Dapper;

var insertCourse = @"
    INSERT INTO courses (course_name, course_description, created_at)
    VALUES (@CourseName, @CourseDescription, @CreatedAt)
    RETURNING id;";

var courseId = connection.QuerySingle<int>(
    insertCourse,
    new
    {
        CourseName = "Course 1",
        CourseDescription = "Course 1 Description",
        CreatedAt = DateTime.Now
    });

var insertSubject = @"
    INSERT INTO subjects (subject_name, subject_description, course_id, created_at)
    VALUES (@SubjectName, @SubjectDescription, @CourseId, @CreatedAt);";

connection.Execute(
    insertSubject,
    new
    {
        SubjectName = "Subject 1",
        SubjectDescription = "Subject 1 Description",
        CourseId = courseId,
        CreatedAt = DateTime.Now
    });

In [ ]:
var selectCourses = @"
    SELECT id, course_name as CourseName, course_description as CourseDescription, 
           created_at as CreatedAt 
    FROM courses;";

var courses = connection.Query<Course>(selectCourses);

courses.DisplayTable();

In [ ]:
// Instead of anonymous object, a concrete object Course could be used as well

connection.Execute(
    insertCourse,
    new Course
    {
        CourseName = "Course",
        CourseDescription = "Course as exact type",
        CreatedAt = DateTime.Now
    });

var courses = connection.Query<Course>(selectCourses);

courses.DisplayTable();

In [ ]:
// Parameters can be also added for select queries

var course1 = connection.Query<Course>(
    @"
    SELECT id, course_name as CourseName, course_description as CourseDescription, 
           created_at as CreatedAt FROM courses WHERE id = @Id;", new { Id = 1 }
);

course1.Display();

#### SQL Injection

Looking at the example above, it might be tempting to just concatenate the SQL query string yourself instead of using parameterized queries.

However this opens up the possibility of SQL inejction attacks. This works by passing such input argument, which the attacker expects to be simply concatenated into SQL query, so that it would break out of original query and do something else on top of it.

Of course it is possible to add validation and workaround these yourself, but the libraries like Dapper already do that out of the box.

In [ ]:
void AddNewCourse(string name, string description)
{
    var insertCourse = @"
        INSERT INTO courses (course_name, course_description, created_at)
        VALUES ('" + name + "', '" + description + "', '" + DateTime.Now + "')";

    Console.WriteLine($"Executing query: [{insertCourse}]");

    connection.Execute(insertCourse);
}

// Everything works if the user input was nice
AddNewCourse("Course aaa", "Desc");
var courses = connection.Query<Course>(selectCourses);
courses.DisplayTable();

In [ ]:
AddNewCourse("Course', 'a', '2025-01-01'); DELETE FROM courses; --", "Desc");
var courses = connection.Query<Course>(selectCourses);
courses.DisplayTable();

### Entity Framework

In Entity Framework everything is going to revolve around the `DbContext` class and `DbSet`s defined in it.

In [ ]:
#r "nuget:Microsoft.EntityFrameworkCore"

// A PostgreSQL adapter for Entity Framework
#r "nuget:Npgsql.EntityFrameworkCore.PostgreSQL"

In [ ]:
public class Course
{
    public int Id { get; set; }
    public string CourseName { get; set; }
    public string CourseDescription { get; set; }
    public DateTime CreatedAt { get; set; }
    public List<Subject> Subjects { get; set; }
}

public class Subject
{
    public int Id { get; set; }
    public string SubjectName { get; set; }
    public string SubjectDescription { get; set; }
    public DateTime CreatedAt { get; set; }

    public int CourseId { get; set; }
        public Course Course { get; set; }
}

In [ ]:
using Microsoft.EntityFrameworkCore;

public class StudiesContext : DbContext
{
    public DbSet<Course> Courses { get; set; }
    public DbSet<Subject> Subjects { get; set; }

    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
    {
        var connectionString = "Host=localhost;Port=5432;Database=example;Username=postgres;Password=devpassword;";

        optionsBuilder.UseNpgsql(connectionString);
    }

    protected override void OnModelCreating(ModelBuilder modelBuilder)
    {
        var courseConfig = modelBuilder.Entity<Course>();
        courseConfig.ToTable("courses");
        courseConfig.HasKey(c => c.Id);
        courseConfig.Property(c => c.Id)
            .HasColumnName("id")
            .ValueGeneratedOnAdd();
        courseConfig.Property(c => c.CourseName)
            .HasColumnName("course_name")
            .HasMaxLength(100)
            .IsRequired();
        courseConfig.Property(c => c.CourseDescription)
            .HasColumnName("course_description");
        courseConfig.Property(c => c.CreatedAt)
            .HasColumnName("created_at")
            .HasDefaultValueSql("CURRENT_TIMESTAMP");

        var subjectConfig = modelBuilder.Entity<Subject>();
        subjectConfig.ToTable("subjects");
        subjectConfig.HasKey(s => s.Id);
        subjectConfig.Property(s => s.Id)
            .HasColumnName("id")
            .ValueGeneratedOnAdd();
        subjectConfig.Property(s => s.SubjectName)
            .HasColumnName("subject_name")
            .HasMaxLength(100)
            .IsRequired();
        subjectConfig.Property(s => s.SubjectDescription)
            .HasColumnName("subject_description");
        subjectConfig.Property(s => s.CreatedAt)
            .HasColumnName("created_at")
            .HasDefaultValueSql("CURRENT_TIMESTAMP");
        subjectConfig.Property(s => s.CourseId)
            .HasColumnName("course_id");
    
        subjectConfig
            .HasOne(s => s.Course)
            .WithMany(c => c.Subjects)
            .HasForeignKey(s => s.CourseId)
            .OnDelete(DeleteBehavior.Cascade);
    }
}

In [ ]:
using System;

var dbContext = new StudiesContext();

var course = new Course
{
    CourseName = "Course 2",
    CourseDescription = "Course 2 Description",
    CreatedAt = DateTime.UtcNow,
    Subjects = new List<Subject>
    {
        new Subject
        {
            SubjectName = "Subject 2",
            SubjectDescription = "Subject 2 Description",
            CreatedAt = DateTime.UtcNow
        }
    }
};

dbContext.Courses.Add(course);
dbContext.SaveChanges();

Console.WriteLine($"✓ Course created with ID: {course.Id}");
Console.WriteLine($"✓ Subject created with ID: {course.Subjects[0].Id}");

In [ ]:
var courses = dbContext.Courses.ToList();
courses.DisplayTable();

In [ ]:
var subjects = dbContext.Subjects.ToList();
subjects.DisplayTable();

#### Eager loading

In [ ]:
// By default related entities are not loaded
// I.e. if I just load courses, their Subjects collection will be null

using (var dbContext = new StudiesContext())
{
    var courses = dbContext.Courses.ToList();
    courses.DisplayTable();
}

In [ ]:
// This is done to prevent loading too much data unintentionally
// i.e. for 1-to-many relationships, we would need 2 loads to get all data

// However if we want to load related entities eagerly, we can use Include
using (var dbContext = new StudiesContext())
{
    var courses = dbContext.Courses.Include(c => c.Subjects).ToList();
    courses.DisplayTable();
}

// If there were even more nested relations, we could use ThenInclude

#### Filtering with EF

In [ ]:
// One of the benefits of EF is that we can continue using C# code for queries
// instead of writing raw SQL

// In this case we would write Linq like expressions, which would then be translated to SQL by EF

using (var dbContext = new StudiesContext())
{
    var courses = dbContext.Courses
        .Where(c => c.CourseName.Contains("C"))
        .OrderBy(c => c.CreatedAt)
        .ToList();

    courses.DisplayTable();
}

In [ ]:
// Queries can be stacked together

using (var dbContext = new StudiesContext())
{
    var courseWithSubject = dbContext.Courses
        .Where(x => x.CourseName.Contains("C"))
        .Where(x => x.Id > 0)
        .ToList();

    courseWithSubject.Display();
}

In [ ]:
// Same Linq style projections and other aggregates can be used as well

using (var dbContext = new StudiesContext())
{
    var courseNames = dbContext.Courses
        .Where(c => c.Id > 0)
        .Select(c => c.CourseName)
        .ToList();

    courseNames.DisplayTable();
}

#### Materilization

In [ ]:
// Important concept to note is that EF does not execute the queries immediately when they are defined.
// Instead, it builds an expression tree that represents the query. The actual execution happens only when
// the results are enumerated, for example, by calling ToList(), FirstOrDefault(), etc.

using (var dbContext = new StudiesContext())
{
    var query = dbContext.Courses
        .Where(c => c.CourseName.Contains("C")); // Nothing is executed in the database yet.

    var courses = query.ToList(); // Execution happens on this statement.

    courses.DisplayTable();
}

In [ ]:
// If possible, it is always better to do filtering and projections on the database side,
// instead of loading all data into memory and then filtering/projection in C# code.

// You can note that the methods that will be executed, are extension methods defined on IQueryable<T> interface.

using (var dbContext = new StudiesContext())
{
    var courses = dbContext.Courses
        .Where(c => c.CourseName.Contains("C")) // Executed in the database
        .Select(x => x.CourseName) // Executed in the database
        .ToArray() // Forces materialization
        .Where(c => c.Length > 0) // Executed in memory
        .ToList();

    courses.DisplayTable();
}

In [ ]:
// Not all of the C# methods can be translated to SQL.
// In such cases, EF will throw an exception at runtime when it tries to execute the query.

bool MyCustomFilter(string courseName)
{
    return courseName.StartsWith("C") && courseName.Length > 5;
}

using (var dbContext = new StudiesContext())
{
    var courses = dbContext.Courses
        .Where(c => MyCustomFilter(c.CourseName)) // Will throw an exception because as far as EF is concerned,
                                                  // this is a random C# method.
        .ToList();

    courses.DisplayTable();
}

In [ ]:
// If the filtering logic cannot be expressed in SQL, you can first materialize the data into memory
// and then apply the custom filter.

using (var dbContext = new StudiesContext())
{
    var courses = dbContext.Courses
        .ToList() // Forces materialization
        .Where(c => MyCustomFilter(c.CourseName)) // Now this works, because it is executed by the C# runtime in memory
        .ToList();

    courses.DisplayTable();
}

### Code first migration in Entity Framework

CLI tool (dotnet tool) will have to be installed to start generating migrations:

```bash
dotnet tool install --global dotnet-ef
```

After the tool is installed, run the command to generate migrations:

```bash
dotnet ef migrations add <name> --project <path to project> --startup-project <path to startup projects> --output-dir <relative path in project dir where to place generated file>
```

After the migration is generated it can be applied by:

1. Scripting the migration and applying it manually: `dotnet ef migrations script`.
1. Applying it by launching the project: `dotnet ef database update`.
1. Applying it automatically via code on project startup: `dbContext.Database.Migrate();`

## Testing code with databases

There are several general approaches how this could be done:

1. In general libraries like `EntityFramework` have In Memory adapters, which stores the data in memory and emulates the usage of real database with having to use one. Pros of such approach is that it is very easy to set up. Cons is that In Memory adapter tries to emulate the real database, but not always succeeds. This failure to correctly replicate the usage of database, can lead to false positives, where tests succeed, but the actual implementation doesn't work. This can lead to false negatives as well, where tests fail, while the implementation is correct.
1. Using Sqlite as database for testing. Sqlite is an "In Application" database that stores all the data in file and, instead of relying on external service for the database engine, can run everything needed in the hosting application. This is a better representation of the actual database and would be more robust approach than using In Memory database. However the problem is still that in real environment different database engine that Sqlite can be used and this would make tests not fully representative of the final solution.
1. Using test containers.

### Testing with test containers

Test containers is a nice option to setup the database for integration tests. It depends on having docker service running on the machine.

Test containers launch a docker container with the desired database. After the test completes, the database container is removed. This allows to run tests against the actual database that is launched and cleared only for the specific test.

Although setting up the test container involves quite some code to do that properly.

You can use the class below as fixture for `xUnit` framework, it assumes that tested project is an ASP.NET application and code first migrations are used.

It depends on 2 packages installed:
- `Testcontainers.MySql` or `Testcontainers.PostgreSQL`
- `Respawn`
- `Microsoft.AspNetCore.Mvc.Testing`

```csharp
public class IntegrationTestFactory : WebApplicationFactory<Program>, IAsyncLifetime
{
    private const string DatabaseName = "db name";
    private const string DatabaseUser = "db user";
    private const string DatabasePassword = "user password";

    private readonly PostgreSQLContainer _container = new PostgreSQLContainer()
        .WithImage("postgresql:<version needed>")
        .WithUsername(DatabaseUser)
        .WithPassword(DatabasePassword)
        .WithDatabase(DatabaseName)
        .WithCleanUp(true)
        .Build();

    public IApplicationDbContext DbContext { get; private set; } = null!;
    private Respawner _respawner = null!;
    private DbConnection _connection = null!;

    public async Task ResetDatabaseAsync()
    {
        await _respawner.ResetAsync(_connection);
    }

    public async Task InitializeAsync()
    {
        await _container.StartAsync();
        var dbContext = Services.GetRequiredService<ApplicationDbContext>();

        DbContext = dbContext;

        _connection = dbContext.Database.GetDbConnection();
        await _connection.OpenAsync();

        _respawner = await Respawner.CreateAsync(
            _connection,
            new RespawnerOptions
            {
                DbAdapter = DbAdapter.PostgreSQL,
                TablesToIgnore =
                [
                    new Table("__efmigrationshistory"),
                    new Table("__EFMigrationsHistory "),
                ],
            }
        );
    }

    public new async Task DisposeAsync()
    {
        await _container.DisposeAsync();
    }

    protected override void ConfigureWebHost(IWebHostBuilder builder)
    {
        var configValues = new Dictionary<string, string?>
        {
            ["Database:Host"] = "localhost",
            ["Database:Port"] = _container.GetMappedPublicPort(5432).ToString(),
            ["Database:User"] = DatabaseUser,
            ["Database:Password"] = DatabasePassword,
            ["Database:Database"] = DatabaseName,
        };
        var configuration = new ConfigurationBuilder().AddInMemoryCollection(configValues).Build();

        builder
            .UseConfiguration(configuration)
            .ConfigureAppConfiguration(configurationBuilder =>
            {
                configurationBuilder.AddInMemoryCollection(configValues);
            });

        // Optionally replace it completely
        // builder.ConfigureTestServices(services =>
        // {
        //     services.RemoveAll<GraderDbContext>();

        //     services.AddDbContext<GraderDbContext>(options =>
        //     {
        //         options.UseMySql(
        //             $"Server=localhost;Port={_container.GetMappedPublicPort(3306)};Database={DatabaseName};User={DatabaseUser};Password={DatabasePassword}",
        //             new MySqlServerVersion(ServerVersion.Parse("9.0.1"))
        //         );
        //     });
        // });
    }
}

[CollectionDefinition(nameof(DatabaseTestCollection))]
public class DatabaseTestCollection : ICollectionFixture<IntegrationTestFactory> { }
```

In the actual test class, this fixture should be used like this:

```csharp
[Collection(nameof(DatabaseTestCollection))]
public class MyIntegrationTest : IAsyncLifetime
{
    private readonly IApplicationDbContext _dbContext;
    private readonly Func<Task> _resetDatabase;

    public MyIntegrationTest(IntegrationTestFactory factory)
    {
        _dbContext = factory.DbContext;
        _resetDatabase = factory.ResetDatabaseAsync;
    }

    // All the tests can go here

    public Task DisposeAsync() => Task.CompletedTask;

    public Task InitializeAsync() => _resetDatabase();
}
```